In [1]:
import numpy as np
import tensorflow as tf
from sklearn import cross_validation, metrics
from memn2n import Memn2n
from load_data import load_data_from_file, convert_data_to_number_list

In [2]:
tf.flags.DEFINE_integer("task_id", 1, "bAbI task id, 1 <= id <= 20")
tf.flags.DEFINE_string("data_dir", "tasks_1-20_v1-2/en/", "Directory containing bAbI tasks")
FLAGS = tf.flags.FLAGS

In [3]:
learning_rate=0.01
epsilon=1e-8
max_grad_norm=40.0
batch_size=32
hops=3
epochs=100
embedding_size=20
memory_size=50
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate,epsilon=epsilon)


def story_to_sentences(story_list):
    sent=[]
    for item in story_list:
        sent+=item
    return sent

train,test=load_data_from_file(FLAGS.data_dir,FLAGS.task_id)
all_data=train+test
vocabulary=reduce(lambda x,y:x|y, (set(story_to_sentences(s)+q+a) for s,q,a in all_data))
vocabulary=sorted(vocabulary)
word_num=dict((word,index+1) for index,word in enumerate(vocabulary))

vocabulary_size=len(word_num)+1
max_story_size=max(map(len, (s for s,_,_ in all_data)))
max_query_size=max(map(len, (q for _,q,_ in all_data)))
max_sentence_size=max(map(len, (story_to_sentences(s) for s,_,_ in all_data)))
max_sentence_size=max(max_sentence_size,max_query_size)
max_memory_size=min(max_story_size,memory_size)

#train_validation_test_split
story,query,answer=convert_data_to_number_list(train,word_num,max_sentence_size,max_memory_size)
story=np.transpose(story,[0,2,1])
train_s,val_s,train_q,val_q,train_a,val_a=cross_validation.train_test_split(story,query,answer,test_size=0.1)
test_s,test_q,test_a=convert_data_to_number_list(test,word_num,max_sentence_size,max_memory_size)

test_s=np.transpose(test_s,[0,2,1])

print("Shape of training data: "+str(train_s.shape))
print("Training size: "+str(train_s.shape[0]))
print("Validation size: "+str(val_s.shape[0]))
print("Testing size: "+str(test_s.shape[0]))

train_answer_label=np.argmax(train_a,1)
validation_answer_label=np.argmax(val_a,1)
test_answer_label=np.argmax(test_a,1)

#set the start and end batch index
batch_index=zip(range(0,train_s.shape[0]-batch_size,batch_size),range(batch_size,train_s.shape[0],batch_size))    

Shape of training data: (900, 56, 10)
Training size: 900
Validation size: 100
Testing size: 1000


In [4]:
#sample=([['sandra','got','the','milk'],['sandra','went','to','office'],['mary','went','to','bedroom'],['sandra','moved','to','kitchen']],['where','was','the','milk'],['kitchen'])
#sample_s,sample_q,sample_a=convert_data_to_number_list([sample],word_num,max_sentence_size,max_memory_size)
#result=[x[0] for x in word_num.items() if x[1]==list(sample_a[0]).index(1)]

In [5]:
#print classification accuracy
with tf.Session() as sess:
    model=Memn2n(batch_size,max_sentence_size,max_memory_size,embedding_size,vocabulary_size,optimizer=optimizer)
    for i in range(epochs):
        np.random.shuffle(batch_index)
        cost_sum=0
        for start,end in batch_index:
            s=train_s[start:end]
            q=train_q[start:end]
            a=train_a[start:end]
            cost=model.train(s,q,a)
            cost_sum+=cost
        if i%10==0:
            train_pre_label=[]
            for start in range(0,train_s.shape[0],batch_size):
                end=start+batch_size
                s=train_s[start:end]
                q=train_q[start:end]
                train_pre_label+=list(model.predict(s,q))
            val_pre_label=model.predict(val_s,val_q)
            train_classification_accuracy=metrics.accuracy_score(np.array(train_pre_label),train_answer_label)
            val_classification_accuracy=metrics.accuracy_score(val_pre_label,validation_answer_label)
            print("------------------***********-----------------------")
            print("This is epoch: "+str(i+1)+" !")
            print("Train accuracy rate is: "+str(train_classification_accuracy))
            print("Validation accuracy rate is: "+str(val_classification_accuracy))
            print("The sum of training loss is: "+str(cost_sum))
    test_pre_label=model.predict(test_s,test_q)
    test_classification_accuracy=metrics.accuracy_score(test_pre_label,test_answer_label)
    print("Test accuracy rate is: "+str(test_classification_accuracy))

------------------***********-----------------------
This is epoch: 1 !
Train accuracy rate is: 0.172222222222
Validation accuracy rate is: 0.22
The sum of training loss is: 1931.03199387
------------------***********-----------------------
This is epoch: 11 !
Train accuracy rate is: 0.801111111111
Validation accuracy rate is: 0.72
The sum of training loss is: 634.978572845
------------------***********-----------------------
This is epoch: 21 !
Train accuracy rate is: 0.992222222222
Validation accuracy rate is: 0.95
The sum of training loss is: 73.1768645942
------------------***********-----------------------
This is epoch: 31 !
Train accuracy rate is: 0.996666666667
Validation accuracy rate is: 1.0
The sum of training loss is: 4.47316182498
------------------***********-----------------------
This is epoch: 41 !
Train accuracy rate is: 0.996666666667
Validation accuracy rate is: 1.0
The sum of training loss is: 17.7311758511
------------------***********-----------------------
This 